In [1]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma
import chromadb
from dotenv import load_dotenv

from function.secrets import secrets

load_dotenv(override=True)

from chromadb.config import Settings


def migrate_chroma_to_faiss(
    chroma_dir: str = "chroma_db", faiss_dir: str = "faiss_db"
) -> None:
    """
    Migrate documents from Chroma to FAISS.

    Parameters
    ----------
    chroma_dir : str
        Directory containing the Chroma database
    faiss_dir : str
        Directory where FAISS database will be saved
    """
    embedding_model = OpenAIEmbeddings(
        model="text-embedding-3-large", api_key=secrets.OPENAI_API_KEY
    )

    chroma_client = chromadb.PersistentClient(
        path=chroma_dir,
        settings=Settings(anonymized_telemetry=False, allow_reset=True),
    )
    chroma_db = Chroma(
        client=chroma_client,
        collection_name="multimodal_docs",
        embedding_function=embedding_model,
        persist_directory=chroma_dir,
    )
    results = chroma_db.get()
    documents = results["documents"]
    metadatas = results["metadatas"]
    ids = results["ids"]

    print(f"Found {len(documents)} documents in Chroma database")

    faiss_db = FAISS.from_texts(
        texts=documents, embedding=embedding_model, metadatas=metadatas, ids=ids
    )

    # Save FAISS database
    faiss_db.save_local(faiss_dir)
    print(f"Successfully migrated and saved FAISS database to {faiss_dir}")


if __name__ == "__main__":
    migrate_chroma_to_faiss()

ModuleNotFoundError: No module named 'langchain_chroma'